## Table of Contents
* [Connecting](#Basics-of-Connecting-to-a-DB2-Database)
    * [Make a Connection](#Make-a-Connection)
        * [Imports](#Imports)
        * [Define Credentials](#Define-Credentials)
        * [Establish Connection](#Establish-Connection)
    * [Retrieve Data](#Retrieve-Data)
        * [Metadata](#Metadata)
    * [Close Connection](#Close-Connection)
* [Creating Tables, Loading, and Querying](#Creating-Tables,-Loading,-and-Querying)
    * [Imports](#Import-Additional-Libraries)
    * [Connect](#Connect-to-Database)
    * [Create Tables](#Create-a-Table-in-the-Database)
        * [Drop Exisiting Table](#Drop-Table-if-it-Already-Exists)
        * [CREATE Query](#Construct-Create-Query)
    * [Insert Data](#Insert-Data)
    * [Query Table](#Query-Table-for-Data)
        * [Fetch Data](#Fetch-Data)
        * [Update Data](#Update-Data)
    * [Retrieve Data into Pandas](#Retrieve-Data-into-Pandas)

# Basics of Connecting to a DB2 Database

## Make a Connection

### Imports

In [1]:
!pip install ibm_db
import ibm_db

### Define Credentials
dsn_hostname = "YourDb2Hostname" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"<br>
dsn_uid = "YourDb2Username"        # e.g. "abc12345"<br>
dsn_pwd = "YourDb2Password"      # e.g. "7dBZ3wWt9XN6$o0J"<br>
dsn_driver = "{IBM DB2 ODBC DRIVER}"<br>
dsn_database = "BLUDB"            # e.g. "BLUDB"<br>
dsn_port = "50000"                # e.g. "50000"<br>
dsn_protocol = "TCPIP"            # i.e. "TCPIP"<br>

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
dsn_hostname = "dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"

### Establish Connection

In [4]:
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

In [5]:
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB on host:  dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net


## Retrieve Data

### Metadata

In [6]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.01.0404
DB_NAME:    BLUDB


In [7]:
#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

DRIVER_NAME:           DB2CLI.DLL
DRIVER_VER:            11.05.0400
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1252
CONN_CODEPAGE:         1208


## Close Connection

In [8]:
ibm_db.close(conn)

True

# Creating Tables, Loading, and Querying

## Import Additional Libraries

In [9]:
import pandas as pd
import ibm_db_dbi

## Connect to Database

In [10]:
dsn_hostname = "dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"

#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB on host:  dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net


## Create a Table in the Database

### Drop Table if it Already Exists (CAUTION: SEE NOTE)

In [11]:
##########################################################################################################
#NOTE The INSTRUCTOR table is our test table, it is okay to drop it to clear it so we can make a clean one

####### ONLY DROP A TABLE IN NON-TESTING CONTEXTS IF YOU ARE 100% CERTAIN IT IS NECESSARY TO DO SO #######

##########################################################################################################

#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

### Construct CREATE Query

In [12]:
#Construct the Create Table DDL statement - replace the ... with rest of the statement
createQuery = "create table INSTRUCTOR(id INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"

#Now fill in the name of the method and execute the statement
#createStmt = ibm_db.replace_with_name_of_execution_method(conn, createQuery)
createStmt = ibm_db.exec_immediate(conn, createQuery)

## Insert Data
Construct the query - replace ... with the insert statement<br>
insertQuery = "..."<br><br>
Execute the insert statement<br>
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [13]:
#Construct the query - replace ... with the insert statement
insertQuery = "INSERT INTO INSTRUCTOR VALUES ('1', 'Rav', 'Ahuja', 'Toronto', 'CA'), ('2', 'Raul', 'Chong', 'Markham', 'CA'), ('3', 'Hima', 'Vasudevan', 'Chicago', 'US')"

#execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

## Query Table for Data

### Fetch Data

In [14]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only)
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'Toronto',
 3: 'Toronto',
 'CCODE': 'CA',
 4: 'CA'}

In [15]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


### Update Data

In [16]:
#Construct the query that updates instructor city
updateQuery = "UPDATE INSTRUCTOR SET CITY='Moosetown' WHERE ID=1"

#Execute the statement
updateStmt = ibm_db.exec_immediate(conn, updateQuery)

## Retrieve Data into Pandas

In [17]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [18]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#retrieve the query results into a pandas dataframe
pdf = pd.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf.LNAME[0]

'Ahuja'

In [19]:
#print the entire data frame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,Moosetown,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [20]:
pdf.shape

(3, 5)

## Close the Connection

In [21]:
ibm_db.close(conn)

True